<a href="https://colab.research.google.com/github/jjhuang4/plant-classification/blob/main/VGG19_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch torchvision

Goal: reproduce results from transfer learning with pretrained VGG-19 classifier on plant dataset

Research paper:
https://arxiv.org/ftp/arxiv/papers/2209/2209.03076.pdf

Dataset:
https://www.cvl.isy.liu.se/en/research/datasets/swedish-leaf/

Medium article explaining how to set up VGG-19: https://medium.com/mlearning-ai/image-detection-using-convolutional-neural-networks-89c9e21fffa3


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models

## Making use of GPU resource

Access GPU in Colab: Runtime -> Change runtime type -> Change to GPU version

See this link for more info https://stackoverflow.com/questions/50560395/how-to-install-cuda-in-google-colab-gpus

In [ ]:
!nvidia-smi

Tue Oct 24 04:20:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

cnn = models.vgg19(pretrained=True).features.to(device).eval()

Using device: cuda

Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:05<00:00, 99.1MB/s]


## Downloading and loading in dataset

Swedish trees dataset separated into 15 classes of plant species, summing to around 1125 images total. Data can be downloaded from zipfiles on the site

In [29]:
import os
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

#cwd = os.getcwd()
#zippath = cwd + "/data"
#zipurl = 'https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip'
#if os.path.exists(zippath) == False:
#    with urlopen(zipurl) as zipresp:
#        with ZipFile(BytesIO(zipresp.read())) as zfile:
#            zfile.extractall(zippath)
#else:
#    print("PennFudanPed dataset already exists in current working directory")

def download_zip(zipurl, folder_name):
  cwd = os.getcwd()
  zippath = cwd + "/data/" + folder_name
  if os.path.exists(zippath) == True:
    print("This dataset already exists in current working directory")
    return
  with urlopen(zipurl) as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            zfile.extractall(zippath)

In [32]:
for num in range(1, 16):
  link = f"https://www.cvl.isy.liu.se/en/research/datasets/swedish-leaf/leaf{num}.zip"
  print(f"Downloading from {link}")
  download_zip(link, "data"+str(num))

This dataset already exists in current working directory
This dataset already exists in current working directory
This dataset already exists in current working directory
This dataset already exists in current working directory
This dataset already exists in current working directory
This dataset already exists in current working directory
This dataset already exists in current working directory
This dataset already exists in current working directory
This dataset already exists in current working directory
This dataset already exists in current working directory
This dataset already exists in current working directory
This dataset already exists in current working directory
This dataset already exists in current working directory
This dataset already exists in current working directory


In [33]:
!touch .gitignore

In [41]:
!cat .gitignore && echo "/data" > .gitignore